In [ ]:
yoi = [2020]
toi = ['15RTN', '15RTP', '14RNS']
root_path = "C:/Users/mcecil/CGA/CDL/"
spath = root_path + f"CDL_HLS_dataframe{yoi[0]}.csv"
url_file = root_path + f'CDL_HLS_dataframe{yoi[0]}.csv'
index_file = root_path + f'mask_index_{yoi[0]}.csv'

image_index_file = root_path + f'image_index_{yoi[0]}.csv'

kml_file = root_path + 'sentinel_tile_grid.kml'
geojson_file = root_path + 'geojson/' + 'aoi.geojson'  ## chip file, lat-long
geojson_rpj_file = root_path + 'aoi_rpj.geojson'
hdf_dir = root_path + 'hdf/'
tiff_dir = root_path + 'tif/'
tiff_dir_rpj = root_path + 'tif_5070_new/'

mask_dir = root_path + 'masks/'

tile_src_path = root_path + "tile_src.csv"

In [ ]:

# Switch plotting backend
import matplotlib
matplotlib.use('agg')

# Import required modules
#import cartopy.crs as ccrs
import os
from datetime import datetime
import pandas as pd
import geopandas
#import pyhdf
import rasterio
import rasterio.mask
import matplotlib.pyplot as pp
import nasa_hls
import numpy as np
import time
import fiona
from pathlib import Path
import time
import urllib.request as urlreq
import shutil
from glob import glob
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [ ]:
# # Load the HLS tiles and place there coordinates into a numpy array for processing later
# fiona.drvsupport.supported_drivers['KML'] = 'rw'
# tile_src = geopandas.read_file(kml_file, driver='KML')
# tile_name = []
# tile_x = []
# tile_y = []
# for tile_ind in range(tile_src.shape[0]):
#     tile_name.append(tile_src.iloc[tile_ind].Name)
#     tile_x.append(tile_src.iloc[tile_ind].geometry.centroid.x)
#     tile_y.append(tile_src.iloc[tile_ind].geometry.centroid.y)
# tile_name = np.array(tile_name)
# tile_x = np.array(tile_x)
# tile_y = np.array(tile_y)
# tile_src = pd.concat([tile_src, tile_src.bounds], axis = 1)
# #del tile_src
# tile_src.head(5)

In [ ]:
# # Load the HLS query csv for this year
# qfn = open(url_file)
# qtile = []
# qyear = []
# qmonth = []
# qday = []
# qdate = []
# qurl = []
# for i, line in enumerate(qfn):
#     if (i == 0): # Skip header
#         continue
#     dummy = line.split(",")
#     qtile.append(dummy[2])
#     qdate.append(dummy[3])
#     qurl.append(dummy[4])
    
#     qday.append(int(dummy[3].split("-")[2]))
#     qmonth.append(int(dummy[3].split("-")[1]))
#     qyear.append(int(dummy[3].split("-")[0]))
# qfn.close()
# qdate = np.array(qdate)
# qtile = np.array(qtile)
# qurl = np.array(qurl)
# qmonth = np.array(qmonth)
# qyear = np.array(qyear)

In [ ]:
# Create the index file
# fn_index = open(index_file, "w")
# fn_index.write('EventID,Name,FireDate,MaskDate,BurnAcres,Tile,MaskFile')
# fn_index.close()
#image_index

Load geojson used for query. 

In [ ]:
aoi_src = geopandas.read_file(geojson_file) ## this could be a list
nfeatures = aoi_src.shape[0]
aoi_src = pd.concat([aoi_src, aoi_src.bounds], axis = 1)
aoi_src['centroid_x'] = aoi_src.centroid.x
aoi_src['centroid_y'] = aoi_src.centroid.y
aoi_src['chip_id'] = range(1, len(aoi_src) + 1)
aoi_src.head(5)

Download HDF

In [ ]:
for aoi_ind in range(nfeatures):
    aoi_row = aoi_src.iloc[aoi_ind]
    print(aoi_row)
    print(aoi_src.bounds)
    aoi_chip_id = int(aoi_row.chip_id)
    aoi_x = float(aoi_row.centroid_x)
    aoi_y = float(aoi_row.centroid_y)
        
        
    # Identify what tile the burn scar is in
    s = (tile_x-aoi_x)**2+(tile_y-aoi_y)**2
    tname = tile_name[np.argmin(s)]
    print(s)
    print(tname)
    
    aoi_src.at[aoi_ind, 'tile'] = tname
   # aoi_src.at[aoi_ind, 'chip_id'] = 'chip_' + str(aoi_chip_id)
    
    
    # Subset potential images based on tile and date
    tile_mask = qtile == tname
    # Subset based on date
    date_mask1 = (qyear == yoi[0])
  #  diff = (qmonth-bs_month)%12
   # date_mask2 = ((diff <= month_threshold[1]) & (diff >= month_threshold[0]))
    date_mask = date_mask1  #& date_mask2
    
    mask = tile_mask & date_mask
    candidate_dates = qdate[mask]
    candidate_tiles = qtile[mask]
    candidate_urls = qurl[mask]
    # Loop through images and check cloud cover
    for cd, ct, cu in zip(candidate_dates, candidate_tiles, candidate_urls):     
        local_name = cu.split('/')[-1].replace("\n", "")
        print(local_name)
        if (local_name != 'HLS.S30.T14RNS.2020040.v1.4.hdf'):
            continue
        # Download a candidate file to check cloud cover
        try:
            urlreq.urlretrieve(cu, filename=hdf_dir+local_name)
        except:
            continue
#aoi_src.head(5)

Extract metadata (WORKING with adjusted function)

In [ ]:
from subprocess import Popen, PIPE

def get_metadata_from_hdf_mine(src, fields=["cloud_cover", "spatial_coverage"]):
    """Get metadata from a nasa-hls hdf file. See HLS user guide for valid fields.
    
    HLS User Guide - see Section 6.6: 
    
    https://hls.gsfc.nasa.gov/wp-content/uploads/2019/01/HLS.v1.4.UserGuide_draft_ver3.1.pdf
    """
    band="QA"
    cmd = f'gdalinfo HDF4_EOS:EOS_GRID:"{src}":Grid:{band}'
    print('cmd')
    print(cmd)
    p = Popen(cmd, stdout=PIPE, shell=True)
    output, err = p.communicate()
    output = str(output)[2:-1].replace("\\n", "\n")
    rc = p.returncode
    metadata = {}
    for line in output.split("\n"):
        for field in fields:
            if field in line:
                metadata[field] = line.split("=")[1].strip()
                try:
                    metadata[field] = float(metadata[field])
                except:
                    pass
    for field in fields:
        if field not in metadata.keys():
            warnings.warn(f"Could not find metadata for field '{field}'.")
    return metadata

In [ ]:
candidate_hdf = sorted(glob(hdf_dir + '*.hdf'))
#print(candidate_hdf)
for name in candidate_hdf:
    local_name = name.replace('\\', '/').split('/')[-1].replace("\n", "")
    print(local_name)
        

In [ ]:
image_index = pd.DataFrame(columns = ['image_id', 'tile', 'date', 'month', 'cloud_coverage', 'spatial_coverage'])

candidate_hdf = sorted(glob(hdf_dir + '*.hdf'))

for img in candidate_hdf:
    print(img)
    
  
    local_name = img.replace('\\', '/').split('/')[-1].replace("\n", "")
    print(local_name)
    # if local_name != "HLS.S30.T14RNS.2020117.v1.4.hdf":
    #     continue
    # Download a candidate file to check cloud cover
   # try:
  #  print('md')
    try:
        print(hdf_dir+local_name)
        md = get_metadata_from_hdf_mine(hdf_dir+local_name)
    except:
        print(img + ' skipped')
        continue
    print(md)
    cloud_cover = int(md['cloud_cover'].replace('\\r', ''))
    print(cloud_cover)
    spatial_coverage = int(md['spatial_coverage'].replace('\\r', ''))
    print(spatial_coverage)
    image_id = local_name.replace('.hdf', '')
    date = local_name.split('.')[3]
    image_date_string = image_id.split('.')[3]
    image_date = pd.to_datetime(image_date_string, format="%Y%j").date()
    image_month = image_date.month


    new_row = pd.DataFrame({'image_id':  [image_id],
               'tile': [tname],
               'date': [image_date],
               'month': [image_month],
               'cloud_coverage': [cloud_cover],
               'spatial_coverage': [spatial_coverage]})
    image_index = pd.concat([image_index, new_row], ignore_index = True)


#image_index.reset_index()
        # except:
        #     print('error')
        #     continue
        
#aoi_src.head(5)

In [ ]:
image_index.head(10)
#image_index.to_csv(image_index_file)

Convert HDF to single-layer TIF (Using HLS library, WORKING with adjusted function)

In [107]:
import logging
from pathlib import Path
from tqdm import tqdm
import subprocess

#from .utils import BAND_NAMES
#from .utils import get_cloud_coverage_from_hdf
BAND_NAMES = {'S30': {'Coastal_Aerosol': 'B01',
                      'Blue': 'B02',
                      'Green': 'B03',
                      'Red': 'B04',
                      'Red_Edge1': 'B05',
                      'Red_Edge2': 'B06',
                      'Red_Edge3': 'B07',
                      'NIR_Broad': 'B08',
                      'NIR_Narrow': 'B8A',
                      'Water_Vapor' : 'B09',
                      'Cirrus': 'B10',
                      'SWIR1': 'B11',
                      'SWIR2': 'B12',
                      'QA': 'QA'},
              'L30': {'Coastal_Aerosol': 'band01',
                      'Blue': 'band02',
                      'Green': 'band03',
                      'Red': 'band04',
                      'NIR': 'band05',
                      'SWIR1': 'band06',
                      'SWIR2': 'band07',
                      'Cirrus': 'band09',
                      'TIRS1': 'band10',
                      'TIRS2': 'band11',
                      'QA': 'QA'}}

log = logging.getLogger(__name__)

def convert_hdf2tiffs_mine(hdf_path, dstdir, dstdir_rpj, bands=None, max_cloud_coverage=100,
                      gdal_translate_options=None, delete_original_tiff = False):
    """Convert (a subset of) hdf-file layers to single layer file GeoTiffs.
    
    Starting with GDAL 3.1 a Cloud Optimized GeoTIFF generator is available.
    It can be used to generate COGs instead of normal GeoTIFFs.
    For building a COG with default options simply use 
    `gdal_translate_options='-of COG'`.
    For more information and options see https://gdal.org/drivers/raster/cog.html.
    """

    if ".L30." in str(hdf_path):
        product = "L30"
    elif ".S30." in str(hdf_path):
        product = "S30"
    else:
        log.exception(f"FATAL ERROR : COULD NOT DERIVE PRODUCT.")
        raise ValueError(f"Could not derive the product ('L30' or 'S30') from the {hdf_path}.")

    if bands is None:
        bands = list(BAND_NAMES[product].keys())

    return_none = False
    for long_band_name in bands:
        if long_band_name not in BAND_NAMES[product].keys():
            continue
        else:
            band = BAND_NAMES[product][long_band_name]

        if not isinstance(hdf_path, str):
            hdf_path_str = str(hdf_path.resolve())
        else:
            hdf_path = Path(hdf_path)
            hdf_path_str = hdf_path

        if max_cloud_coverage < 100:
            try:
                cc = get_cloud_coverage_from_hdf(hdf_path_str)
                log.debug(f"DERIVED CLOUD COVER: {cc}")
            except:
                log.exception(f"COULD NOT DERIVE CLOUD COVER => PROCESSING ANYWAY: {hdf_path_str}")
                cc = 0

            if cc > max_cloud_coverage:
                log.debug(f"SKIPPING CONVERSION - TOO HIGH CLOUD COVER: {cc}")
                return_none = True
                break

        dst = (dstdir / hdf_path.stem).resolve() / f"{hdf_path.stem}__{long_band_name}.tif"
        dst_rpj = (dstdir_rpj / hdf_path.stem).resolve() / f"{hdf_path.stem}__{long_band_name}__5070.tif"
      #  print('test2')
      #  print(dst)
      #  print(dst_rpj)

        if not dst.exists():
            print('creating files')
            cmd = f'gdal_translate HDF4_EOS:EOS_GRID:"{hdf_path_str}":Grid:{band} {dst}'
            if gdal_translate_options:
                cmd += f" {gdal_translate_options}"
            log.debug(f"CMD: {cmd}")
            dst.parent.mkdir(exist_ok=True, parents=True)
            try:
                subprocess.check_call(cmd, shell=True)
            except Exception:
                log.exception(f"ERROR DURING CONVERSION WITH CMD: {cmd}.")
            ## reprojection
          #  gdalwarp -t_srs EPSG:5070 -r bilinear -tr 30 -30 <input_file> <output_file>
            cmd = f"gdalwarp -t_srs EPSG:5070 -r bilinear -tr 30 -30 {dst} {dst_rpj}"
          #  if gdal_translate_options:
           #     cmd += f" {gdal_translate_options}"
            log.debug(f"CMD: {cmd}")
            dst_rpj.parent.mkdir(exist_ok=True, parents=True)
            try:
                subprocess.check_call(cmd, shell=True)
            except Exception:
                log.exception(f"ERROR DURING REPROJECTION WITH CMD: {cmd}.")   
    
    if delete_original_tiff:
        shutil.rmtree((dstdir / hdf_path.stem).resolve())
                
            
    if return_none:
        return None
    else:
        return dstdir / hdf_path.stem

In [108]:
image_index = pd.read_csv(image_index_file)
image_index.head(5)

,Unnamed: 0,image_id,tile,date,month,cloud_coverage,spatial_coverage,convert_to_tif
0,0,HLS.S30.T14RNS.2020002.v1.4,14RNS,2020-01-02,1,100,41,False
1,1,HLS.S30.T14RNS.2020005.v1.4,14RNS,2020-01-05,1,0,100,False
2,2,HLS.S30.T14RNS.2020007.v1.4,14RNS,2020-01-07,1,29,41,False
3,3,HLS.S30.T14RNS.2020010.v1.4,14RNS,2020-01-10,1,89,100,False
4,4,HLS.S30.T14RNS.2020012.v1.4,14RNS,2020-01-12,1,71,41,False


In [109]:
cloud_threshold = 80
valid_months = [4, 5, 6, 7, 8, 9]
for k in range(len(image_index)):
    img_data = image_index.iloc[k]
    # Loop through images and check cloud cover
    image_id = img_data['image_id']
    print(image_id)
    tile = img_data['tile']
    date = img_data['date']
    month = img_data['month']
    cloud_cov = img_data['cloud_coverage']
    
    if((cloud_cov <= cloud_threshold) and (month in valid_months)):
        print('valid')
        print(image_id)
        convert_hdf2tiffs_mine(Path(hdf_dir + image_id + '.hdf'), Path(tiff_dir), Path(tiff_dir_rpj), delete_original_tiff = False)
        image_index.at[k, 'convert_to_tif'] = True
    else:
        print('invalid')
    #    print(image_id)
        image_index.at[k, 'convert_to_tif'] = False
#     for cd, ct, cu in zip(candidate_dates, candidate_tiles, candidate_urls):
#         print(cu)
#         local_name = cu.split('/')[-1].replace("\n", "")
        
#         if local_name != "HLS.S30.T14RNS.2020005.v1.4.hdf":
#             continue
#        # print('test')
#         #print(Path(hdf_dir+local_name))
#         #print(Path(tiff_dir))
#         convert_hdf2tiffs_mine(Path(hdf_dir+local_name), Path(tiff_dir), Path(tiff_dir_rpj), delete_original_tiff = False)

HLS.S30.T14RNS.2020002.v1.4
invalid
HLS.S30.T14RNS.2020005.v1.4
invalid
HLS.S30.T14RNS.2020007.v1.4
invalid
HLS.S30.T14RNS.2020010.v1.4
invalid
HLS.S30.T14RNS.2020012.v1.4
invalid
HLS.S30.T14RNS.2020015.v1.4
invalid
HLS.S30.T14RNS.2020017.v1.4
invalid
HLS.S30.T14RNS.2020040.v1.4
invalid
HLS.S30.T14RNS.2020042.v1.4
invalid
HLS.S30.T14RNS.2020047.v1.4
invalid
HLS.S30.T14RNS.2020057.v1.4
invalid
HLS.S30.T14RNS.2020067.v1.4
invalid
HLS.S30.T14RNS.2020070.v1.4
invalid
HLS.S30.T14RNS.2020072.v1.4
invalid
HLS.S30.T14RNS.2020075.v1.4
invalid
HLS.S30.T14RNS.2020077.v1.4
invalid
HLS.S30.T14RNS.2020080.v1.4
invalid
HLS.S30.T14RNS.2020082.v1.4
invalid
HLS.S30.T14RNS.2020087.v1.4
invalid
HLS.S30.T14RNS.2020097.v1.4
invalid
HLS.S30.T14RNS.2020100.v1.4
invalid
HLS.S30.T14RNS.2020102.v1.4
invalid
HLS.S30.T14RNS.2020107.v1.4
valid
HLS.S30.T14RNS.2020107.v1.4
creating files
creating files
creating files
creating files
creating files
creating files
creating files
creating files
creating files
creating fi

KeyboardInterrupt: 

Create mask tif files (1 if valid data, 0 if not). Uses geojson (lat long) to mask. Does not crop. (WORKS)

In [ ]:
image_index.head(25)
image_index.to_csv(image_index_file)

In [ ]:
# for aoi_ind in range(nfeatures):
#     aoi_row = aoi_src.iloc[aoi_ind]
#     print(aoi_row)
#     print(aoi_src.bounds)
#     aoi_x = float(aoi_row.centroid_x)
#     aoi_y = float(aoi_row.centroid_y)
        
#     # Identify what tile the burn scar is in
#     s = (tile_x-aoi_x)**2+(tile_y-aoi_y)**2
#     tname = tile_name[np.argmin(s)]
#     print(s)
#     print(tname)
#     aoi_src.at[aoi_ind, 'tile'] = tname
#     aoi_src.at[aoi_ind, 'chip_id'] = 'chip_' + str(aoi_ind)
    
#     # Subset potential images based on tile and date
#     tile_mask = qtile == tname
#     # Subset based on date
#     date_mask1 = (qyear == yoi[0])
#   #  diff = (qmonth-aoi_month)%12
#    # date_mask2 = ((diff <= month_threshold[1]) & (diff >= month_threshold[0]))
#     date_mask = date_mask1  #& date_mask2
    
#     mask = tile_mask & date_mask
#     candidate_dates = qdate[mask]
#     candidate_tiles = qtile[mask]
#     candidate_urls = qurl[mask]
#     # Loop through images and check cloud cover
#     for cd, ct, cu in zip(candidate_dates, candidate_tiles, candidate_urls):
#        # print(cu)
#         local_name = cu.split('/')[-1].replace("\n", "")
#         print(local_name)
#         # if local_name != "HLS.S30.T14RNS.2020345.v1.4.hdf":
#         #     continue
#         # print('valid')
#         # Download a candidate file to check cloud cover
#         # try:
#         #     urlreq.urlretrieve(cu, filename=hdf_dir+local_name)
#         # except:
#         #     continue
    
#         # Check cloud cover and spacial coverage
#       #  try:
#             # md = nasa_hls.get_metadata_from_hdf(hdf_dir+local_name, fields=['cloud_cover', 'spatial_coverage'])
#             # if ((md["cloud_cover"] < cldfrac_threshold) and (md['spatial_coverage'] > spacecov_threshold)):
            
#                 # Convert to a geotiff            
#                 # nasa_hls.convert_hdf2tiffs(Path(hdf_dir+local_name), Path(tiff_dir))

#             # Open the new geotiff
#         date = datetime.strptime(cd, '%Y-%m-%d')
#         doy = (date-datetime(date.year, 1, 1)).days+1
#         fp = f'HLS.S30.T{ct}.{date.year}{doy:03d}.v1.4'
#         src = rasterio.open(f'{tiff_dir}/{fp}__B04.tif')
#         red = src.read(1)
#         print(src)
#         print(red)

#         # Open NIR too to compute NDVI
#         nir_src = rasterio.open(f'{tiff_dir}/{fp}__B08.tif')
#         nir = nir_src.read(1)

        
#         qa_src = rasterio.open(f'{tiff_dir}/{fp}__QA.tif')
#         qa = qa_src.read(1)
        
#         print(qa_src)
#         print(qa)
        
#         # Compute NDVI
#         red = np.clip(red/15000, 0, 1)
#         nir = np.clip(nir/15000, 0, 1)
#         ndvi = np.clip((nir-red)/(nir+red), 0, 1)

#         # Convert burn scar shape to proper geometry
#         aoi_shape = aoi_src.to_crs(src.crs).iloc[aoi_ind]

#         print(aoi_shape)
#         # Build the mask
#         out_image, out_transform = rasterio.mask.mask(src, [aoi_shape.geometry], crop=False)
#         print(np.min(out_image), np.max(out_image))
#         out_image = np.clip(out_image, 0, 1)
#         print(np.min(out_image), np.max(out_image))

#         out_meta = src.meta
#         out_meta.update({'driver':'GTiff', 'height':out_image.shape[1],
#             'width':out_image.shape[2], 'transform':out_transform})

#         with rasterio.open(f'{mask_dir}/{fp}.mask.tif', 'w', **out_meta) as dest:
#             dest.write(out_image)

#         # Plot the original and mask
#         fig, [ax1, ax2] = pp.subplots(ncols=2)
#         ax1.imshow(ndvi, cmap='YlGn', vmin=0, vmax=1)
#         ax2.imshow(np.squeeze(out_image), cmap='Greys')
#         pp.savefig(f'{mask_dir}/{fp}.mask.jpg')
#         pp.close()

#         # Save in index file
# #         fn_index = open(index_file, 'a')
# #         fn_index.write(f'\n{aoi_row.Event_ID},{aoi_row.Incid_Name},{aoi_row.Ig_Date},{cd},{aoi_row.BurnBndAc},{ct},{fp}.mask.tif')
# #         fn_index.close()

# #         # Move onto the next burn scar
# #         del src
# #         del nir_src
#        # break
#         # except:
#         #     print('error')
#         #     continue